In [1]:
import json
import pandas as pd
from traces import *
from observation import *
from os import read
from extract.model import Model
import os
from planner import PseudoPlanner


In [2]:

def read_json_file(read_only):
    json_file_path = "../data/traces_plan_r1_no_obj_lim.json"
    with open(json_file_path, 'r') as file:
        data = json.load(file)
        return data

In [3]:



def run_single_experiment(learning_obj,dir, debug):
    res = []
    domain = learning_obj['domain']
    raw_traces = learning_obj['traces']
    lo_id = learning_obj['id']
    traces = []
    for raw_trace in raw_traces:
        steps = []
        for i, raw_step in enumerate(raw_trace):
            action_name = raw_step['action']
            obj_names = raw_step['objs']
            objs = []
            for obj in obj_names:
                obj_name, obj_type = obj.split("?")
                objs.append(PlanningObject(obj_type, obj_name))
            action = Action(action_name, objs)
            step = Step(State(), action, i)
            steps.append(step)
        trace = Trace(steps)
        traces.append(trace)

    tracelist = TraceList(traces)
    obs_tracelist = tracelist.tokenize(ActionObservation, ObservedTraceList)
    
    # dods = [0, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1]
    dods = [0.1]
    for dod in dods:
        if (dod == 0):
            dod_str = "0.0"
        elif (dod == 1):
            dod_str = "1.0"
        else:
            dod_str = str(dod)
        domain_filename = dir + domain + "_0_" + str(lo_id) + "_dod" + dod_str + ".pddl"
        if not os.path.exists(domain_filename):
            print("File {0} does not exist".format(domain_filename))
            res.append([lo_id, dod, domain, -1, -1])
            continue
        gt_domain_filename = "../data/goose-benchmarks/tasks/{0}/domain.pddl".format(domain)
        try:
            pp = PseudoPlanner(domain_filename, 'twoway', gt_domain_filename)
            e1s = []
            e2s = []
            for trace in obs_tracelist:
                actions = [step.action for step in trace]
                e1, e2 = pp.check_executability(actions, debug)
                e1s.append(e1)
                e2s.append(e2)
            exe1 = sum(e1s)/len(e1s)
            exe2 =  sum(e2s)/len(e2s)
            res.append([lo_id, dod, domain, exe1, exe2])
        except Exception as e:
            print("Error in domain {0} with dod {1}: {2}".format(domain, dod, e))
            res.append([lo_id, dod, domain, -1, -1])
    
    return res


In [4]:
read_only = "satellite"
# read_only = None
data = read_json_file(read_only)
methods = ["polocm", "baseline"]
res = []
# for method in methods:
#     dir =  "../experiments/{m}/pddl/".format(m=method)
#     for lo in data:
#         lo_res = run_single_experiment(lo,dir, None)
#         res.extend(lo_res)
# headers = ['lo_id', 'dod', 'domain', 'exe1', 'exe2']
# df = pd.DataFrame(res, columns=headers)
# df.to_csv("../experiments/twoway_exe.csv", index=False)




In [5]:
example = data[0]
dir =  "../experiments/baseline/pddl/"

res = run_single_experiment(example,dir, debug= True)
print(res)

defaultdict(<class 'set'>, {'s0': {'zero'}, 's2': {'robot1'}, 's4': {'tile_0_5'}, 's1': {'tile_2_6', 'tile_4_6', 'tile_0_6', 'tile_1_6', 'tile_3_6'}, 's3': {'black', 'white'}})
number of grounded actions: 109
Grounded actions: [<PropositionalAction '(paint_up zero robot1 tile_2_6 tile_2_6 black)' at 0x7af466df6000>, <PropositionalAction '(paint_up zero robot1 tile_2_6 tile_2_6 white)' at 0x7af466c592e0>, <PropositionalAction '(paint_up zero robot1 tile_2_6 tile_4_6 black)' at 0x7af466c5abd0>, <PropositionalAction '(paint_up zero robot1 tile_2_6 tile_4_6 white)' at 0x7af466c5be90>, <PropositionalAction '(paint_up zero robot1 tile_2_6 tile_0_6 black)' at 0x7af466c61220>, <PropositionalAction '(paint_up zero robot1 tile_2_6 tile_0_6 white)' at 0x7af466c7cdd0>, <PropositionalAction '(paint_up zero robot1 tile_2_6 tile_1_6 black)' at 0x7af466c7d550>, <PropositionalAction '(paint_up zero robot1 tile_2_6 tile_1_6 white)' at 0x7af466c7de80>, <PropositionalAction '(paint_up zero robot1 tile_2_6

In [6]:
# statics = ['childsnack', 'floortile', 'miconic', 'rovers', 'satellite', 'sokoban', 'spanner', 'transport']
# headers = ["domain", "exe", "zero_sort_removed", "static_conditions"]
# for i,learn_obj in enumerate(data):
    
#     exe = run_single_experiment(learn_obj, 0.1, None)